In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
cols = ['short_name', 'station_id', 'ntaname', 'ntacode']
df = pd.read_pickle('../temp/stations_with_ntas.pickle')[cols]
df

,short_name,station_id,ntaname,ntacode
0,6926.01,72,Clinton,MN15
1,5430.08,79,SoHo-TriBeCa-Civic Center-Little Italy,MN24
2,5167.06,82,Chinatown,MN27
3,4354.07,83,Fort Greene,BK68
4,6148.02,116,Hudson Yards-Chelsea-Flatiron-Union Square,MN13
...,...,...,...,...
1580,6432.11,4675,Murray Hill-Kips Bay,MN20
1581,6432.10,4676,Murray Hill-Kips Bay,MN20
1582,4522.07,4677,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,BK38
1583,8616.06,4678,Washington Heights North,MN35


In [3]:
rides = pd.read_pickle("../temp/merged_prepped.pickle")
rides['start_station_id'] = rides['start_station_id'].astype('str')

In [93]:
merged = rides.merge(df, left_on="start_station_id", right_on="station_id", how='left')

In [111]:
stations_per_nta = merged[['ntacode', 'start_station_id']].groupby('ntacode')[['start_station_id']].nunique().reset_index().set_index('ntacode').rename({'start_station_id': 'stations_count'}, axis=1)
stations_per_nta

,stations_count
ntacode,
BK09,11
BK32,2
BK33,33
BK35,9
BK37,31
...,...
QN68,15
QN70,22
QN71,10


In [128]:
# Given df of rides with NTA's joined, count the number of rides and rank them within each group.
def rank_stations_by_nta(df, df_stations_per_nta):
    stations_ranked_by_nta = (df[['uuid', 'ntacode', 'start_station_id']]
        .groupby(['ntacode', 'start_station_id'])
        .count()
        .sort_values(by=['ntacode', 'uuid'], ascending=False)
        .groupby(['ntacode'])
        .rank(method='dense', ascending=False, pct=False)
        .reset_index()
        .rename({'uuid': 'station_rank'}, axis=1)
        .merge(df_stations_per_nta, on='ntacode', how='left')
        .set_index('start_station_id')
    )

    return stations_ranked_by_nta

rank_stations_by_nta(merged, stations_per_nta)

,ntacode,station_rank,stations_count
start_station_id,,,
3514,QN99,1.0,2
3532,QN99,2.0,2
3526,QN72,1.0,10
3524,QN72,2.0,10
3513,QN72,3.0,10
...,...,...,...
143,BK09,7.0,11
3242,BK09,8.0,11
216,BK09,9.0,11
